In [1]:
import pandas as pd

In [2]:
df= pd.read_csv("../dataset/240128_TripAdvisorHotelReviews/dataset_removed_stopwords_ver.csv")

In [3]:
type(df['tokens'][0])

str

In [4]:
##!! 저장을 해도 어차피 다시 list로 변환을 해주어야 한다. 

import pandas as pd
import ast

# 문자열을 파싱하여 리스트로 변환하는 함수
def parse_string_to_list(tokens_str):
    return ast.literal_eval(tokens_str)

# 모든 행에 대해 함수 적용
df['tokens'] = df['tokens'].apply(parse_string_to_list)

In [5]:
sentences = df['tokens']

In [8]:
sentences[0]

['everyone',
 'alby',
 'made',
 'reservation',
 'deenny',
 'greeted',
 'every',
 'morning',
 'hailed',
 'cabs',
 'jennifer',
 'front',
 'desk',
 'made',
 'sure',
 'stay',
 'wonderful',
 'already',
 'rebooked',
 'extended',
 'stay',
 'next',
 'month',
 'work',
 'takes',
 'back',
 'thanks']

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# 예시 데이터 (refined_df['tokens']에 토큰이 이미 들어가 있다고 가정)
# 토큰화된 데이터가 저장된 컬럼 이름이 'tokens'일 경우:
tokenized_text = df['tokens']

2024-02-16 17:06:13.219929: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
tokenized_text[0]

['everyone',
 'alby',
 'made',
 'reservation',
 'deenny',
 'greeted',
 'every',
 'morning',
 'hailed',
 'cabs',
 'jennifer',
 'front',
 'desk',
 'made',
 'sure',
 'stay',
 'wonderful',
 'already',
 'rebooked',
 'extended',
 'stay',
 'next',
 'month',
 'work',
 'takes',
 'back',
 'thanks']

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# 예시 데이터 (refined_df['tokens']에 토큰이 이미 들어가 있다고 가정)
# 토큰화된 데이터가 저장된 컬럼 이름이 'tokens'일 경우:
tokenized_text = df['tokens']

# 토크나이저 객체 생성
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_text)

# 토큰들을 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(tokenized_text)

# 시퀀스 패딩
max_sequence_length = 200  # 시퀀스의 최대 길이 (임의 설정)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [9]:
from gensim.models import FastText

# 토큰화된 문장으로 FastText 모델 학습
model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4)

# 단어 임베딩
embedding_dim = 300
embedded_sequences = [[model.wv[word] for word in sequence] for sequence in sentences]


In [10]:
# 패딩 처리
padded_sequences = pad_sequences(embedded_sequences, padding='post', maxlen=200)

In [11]:
import numpy as np

labels = df['ratings']

# labels 배열
labels = np.array(labels)

# 값을 숫자로 변환하는 코드
labels = np.array([1 if value == 'high' else 0 for value in labels])

print("Modified labels:", labels)

Modified labels: [1 1 1 ... 0 0 0]


In [12]:
import numpy as np

# 넘파이 배열로 변환
embedded_sequences = np.array(padded_sequences)
labels = np.array(labels)

## Train-Test 데이터셋 분리

In [13]:
from sklearn.model_selection import train_test_split

# PyTorch DataLoader로 사용할 데이터셋 클래스를 정의하고 데이터를 생성하는 작업이 필요

# 예시: X는 embedded_sequence, y는 refined_df['ratings']
X_train, X_temp, y_train, y_temp = train_test_split(embedded_sequences, labels, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [14]:
# 결과 확인
print("Train data shapes:", X_train.shape, y_train.shape)
print("Validation data shapes:", X_val.shape, y_val.shape)
print("Test data shapes:", X_test.shape, y_test.shape)

Train data shapes: (7000, 200, 100) (7000,)
Validation data shapes: (1500, 200, 100) (1500,)
Test data shapes: (1500, 200, 100) (1500,)


In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# 데이터셋을 플랫하게 펼침
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat = X_val.reshape(X_val.shape[0], -1)
X_test_flat = X_val.reshape(X_val.shape[0], -1)

## 모델 학습 및 평가

In [16]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# 모델 초기화
model = LogisticRegression()

# 모델 학습
model.fit(X_train_flat, y_train)

/Users/songye/opt/anaconda3/envs/aiffel2/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [17]:
# 예측
y_pred = model.predict(X_val_flat)

# 정확도 평가
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy}")

Validation Accuracy: 0.7033333333333334
